# MNIST Combiner

Combines two models, an SKLearn model and a Tensorflow model for MNIST. The combination does a simple average of the two models.

In [ ]:
!pygmentize MnistCombiner.py

## Start Minikube

In [ ]:
!minikube start --memory 4096

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

## Build Combiner image

In [ ]:
!eval $(minikube docker-env) && s2i build -E environment_rest . seldonio/seldon-core-s2i-python36:0.5 seldonio/mnistcombiner_rest:0.1

## Install Helm

In [ ]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

## Install Seldon Core

In [ ]:
!kubectl create namespace seldon

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd

In [ ]:
!helm install ../../../helm-charts/seldon-core --name seldon-core --namespace seldon \
    --set ambassador.enabled=true

In [ ]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

To send requests to Ambassador ingress in another terminal run:
    
```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8002:8080
```

In [ ]:
%matplotlib inline
import utils
from visualizer import get_graph
mnist = utils.download_mnist()

In [ ]:
get_graph("mnist_combiner.json")

In [ ]:
!pygmentize mnist_combiner.json

In [ ]:
!kubectl apply -f mnist_combiner.json

In [ ]:
!kubectl rollout status deploy/mnistcombo-mnistcombo-3715bc4

In [ ]:
utils.predict_rest_mnist(mnist,"mnistcombo")